This file is **very similar** to Preliminary-analysis-1.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('consolidated-2.csv')

In [3]:
# This is the time in seconds taken to complete a call(included post-processing).
data['Avg-time'] = data['Avg-time'].fillna(0) 

In [4]:
data['lambda'] = data['ncalls'].rolling(window=6).mean() # Number of calls per 5 minutes.

In [5]:
data.set_index('Create-time', inplace=True)

In [6]:
data['mu'] = data['Avg-time']/300 # The term lambda/mu must be unit less.

In [7]:
data['E'] = data['lambda']*data['mu'] # Ordered traffic in erlangs

In [10]:
def factorial(n):
    if n == 0:
        return 1.0
    else:
        f = 1.0
        for m in range(1, n+1):
            f = f * m
        
        return f
    
def prob_of_wait(ncalls, E, m):
    """
    ncalls: number of calls in an interval of 5 minutes.
    E:      traffic
    m:      # agents.
    drop:   a variable that corrects the number of people on the floor. 
    Reference: https://en.wikipedia.org/wiki/Erlang_(unit)
    """
    if ncalls == 0:
        return 0
    
    if m == 0:
        m = 1
        
    x = m - E
    if np.abs(x) < 1e-6:
        x = 0.01
           
    numerator = E**m/factorial(m) * m/x
    denominator = 0
        
    for i in range(m):
        denominator += E**i/factorial(i)
        
    denominator += numerator
    
    p = numerator/denominator
    
    # TODO: Find out when p exceeds 1. Which assumptions of Erlang C formula are violated?
    if p > 1:
        p = 1
        
    return p

def calculate_ASA(wait_prob, N, mu, rho):
    return wait_prob/(N * mu * rho)

In [11]:
data['wait-prob'] = data.apply(lambda r: prob_of_wait(r['ncalls'], r['E'], r['nagents']), axis=1)

In [13]:
data['asa'] = data['wait-prob'] * data['mu']/(data['nagents'] - data['E']) * 5 # Convert to minutes'

In [14]:
data.reset_index(level=0, inplace=True)

In [15]:
data.to_csv('wait_prob_1.csv', index=False)

The next step is to build a time-series model for $E$.